# Chains

Complex applications require chaining LLMs either with each other or with other components.

## Setup
#### Follow [README](https://github.com/tirtho/open-ai/blob/main/README.md) and perform setup before running the notebooks

Reference : 
- [Azure Open AI](https://learn.microsoft.com/en-us/azure/cognitive-services/openai/overview)
- [LangChain home page](https://python.langchain.com/docs/get_started/introduction.html)

#### Load the API key and relevant Python libaries.

In [1]:
import openai
import sys

from azure_openai_setup import set_openai_config, get_openai_global_config_parameters 

set_openai_config()

theOpenAIParams, modelName, modelDeploymentName = get_openai_global_config_parameters()

Got Azure OpenAI Credentials from Azure Key Vault with Azure CLI Auth


## Chain Simple example
Using LLMChain, which is the most basic block chain.

- Takes a prompt template, 
- Formats it with user input.
- Returns response from an LLM

In [2]:
from langchain.chat_models import AzureChatOpenAI

# The openai.<variables> are already filled up by the above 
# set_openai_config() helper function called from above cell.
# Check that function for more details

azureChatClient = AzureChatOpenAI(
            openai_api_key = theOpenAIParams.api_key,
            openai_api_base = theOpenAIParams.api_base,
            openai_api_version = theOpenAIParams.api_version,
            deployment_name = modelDeploymentName,
            temperature=0.9
)

In [5]:
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate

template = """
Generate a single line marketing campaign for the {product} \
of a {company} \
"""

prompt = PromptTemplate(
                input_variables = ['product', 'company'],
                template = template
            )
chain = LLMChain(llm = azureChatClient, prompt = prompt)
print(chain.run({
    'company': 'Property & Casualty Insurance company',
    'product': 'auto insurance'
}))


"Protect your ride and your peace of mind with our comprehensive auto insurance coverage."


## Sequential Chain

More complex chains that involve multiple inputs and multiple final outputs

In [15]:
# This is an LLMChain to write a synopsis given a title of a play and the era it is set in.
template = """You are a playwright. Given the title of play and the era it is set in, it is your job to write a synopsis for that title.

Title: {title}
Era: {era}
Playwright: This is a synopsis for the above play:"""
prompt_template = PromptTemplate(input_variables=["title", 'era'], template=template)
synopsis_chain = LLMChain(llm=azureChatClient, prompt=prompt_template, output_key="synopsis")

In [17]:
# This is an LLMChain to write a review of a play given a synopsis.
template = """You are a play critic from the New York Times. Given the synopsis of play, it is your job to write a review for that play.

Play Synopsis:
{synopsis}
Review from a New York Times play critic of the above play:"""
prompt_template = PromptTemplate(input_variables=["synopsis"], template=template)
review_chain = LLMChain(llm=azureChatClient, prompt=prompt_template, output_key="review")

In [18]:
# This is the overall chain where we run these two chains in sequence.
from langchain.chains import SequentialChain
overall_chain = SequentialChain(
    chains=[synopsis_chain, review_chain],
    input_variables=["era", "title"],
    # Here we return multiple variables
    output_variables=["synopsis", "review"],
    verbose=True)

In [19]:
overall_chain({"title":"Tragedy at sunset on the beach", "era": "Victorian England"})



> Entering new  chain...

> Finished chain.


{'title': 'Tragedy at sunset on the beach',
 'era': 'Victorian England',
 'synopsis': "In the height of Victorian England, a young noblewoman named Felicity falls deeply in love with a commoner, William, who works as a fisherman on the coast. Despite the disapproval of her family and society, Felicity follows her heart and runs away with William to start a new life together. They settle in a small seaside town where they build a new home and start a family. \n\nHowever, their perfect life is soon shattered when a terrible tragedy strikes: while Felicity and William's children are playing on the beach at sunset, a sudden storm brews and their youngest child is swept away by the raging waves. In their grief and desperation to find their child, Felicity and William's relationship begins to unravel, and they are forced to confront the reality of their vastly different backgrounds and the societal pressures that have plagued their relationship from the start. \n\nAs their tragedy unfolds, t

## Router Chain

Create a chain that dynamically selects the next chain to use for a given input.

Router chains are made up of two components:

- The RouterChain itself (responsible for selecting the next chain to call)
- destination_chains: chains that the router chain can route to


#### Create the prompts, templates and the destination_chains

In [6]:
from langchain.chains.router import MultiPromptChain
from langchain.llms import OpenAI
from langchain.chains import ConversationChain
from langchain.chains.llm import LLMChain
from langchain.prompts import PromptTemplate


physics_template = """You are a very smart physics professor. \
You are great at answering questions about physics in a concise and easy to understand manner. \
When you don't know the answer to a question you admit that you don't know.

Here is a question:
{input}"""


math_template = """You are a very good mathematician. You are great at answering math questions. \
You are so good because you are able to break down hard problems into their component parts, \
answer the component parts, and then put them together to answer the broader question.

Here is a question:
{input}"""

In [7]:
prompt_infos = [
    {
        "name": "physics",
        "description": "Good for answering questions about physics",
        "prompt_template": physics_template,
    },
    {
        "name": "math",
        "description": "Good for answering math questions",
        "prompt_template": math_template,
    },
]

In [8]:
destination_chains = {}
for p_info in prompt_infos:
    name = p_info["name"]
    prompt_template = p_info["prompt_template"]
    prompt = PromptTemplate(template=prompt_template, input_variables=["input"])
    chain = LLMChain(llm=azureChatClient, prompt=prompt)
    destination_chains[name] = chain
default_chain = ConversationChain(llm=azureChatClient, output_key="text")

#### Create the LLMRouterChain to determine how to route

In [10]:
from langchain.chains.router.llm_router import LLMRouterChain, RouterOutputParser
from langchain.chains.router.multi_prompt_prompt import MULTI_PROMPT_ROUTER_TEMPLATE

destinations = [f"{p['name']}: {p['description']}" for p in prompt_infos]
destinations_str = "\n".join(destinations)
router_template = MULTI_PROMPT_ROUTER_TEMPLATE.format(destinations=destinations_str)
router_prompt = PromptTemplate(
    template=router_template,
    input_variables=["input"],
    output_parser=RouterOutputParser(),
)
router_chain = LLMRouterChain.from_llm(azureChatClient, router_prompt)

In [11]:
chain = MultiPromptChain(
    router_chain=router_chain,
    destination_chains=destination_chains,
    default_chain=default_chain,
    verbose=True,
)

#### Run the user input against the router now!

In [12]:
print(chain.run("What is black body radiation?"))



> Entering new  chain...
physics: {'input': 'What is black body radiation?'}
> Finished chain.
Black body radiation refers to the electromagnetic radiation emitted by a perfect black body, which is an object that absorbs all radiation that falls on it and emits radiation with a characteristic thermal spectrum. The spectrum of black body radiation depends only on the temperature of the object and is described by Planck's law. Black body radiation is important in many areas of physics, including thermodynamics, quantum mechanics, and astrophysics. It also has practical applications, such as in the design of incandescent light bulbs and infrared cameras.


In [13]:
print(
    chain.run(
        "What is the first prime number greater than 40 such that one plus the prime number is divisible by 3"
    )
)



> Entering new  chain...
math: {'input': 'What is the first prime number greater than 40 such that one plus the prime number is divisible by 3'}
> Finished chain.
?

To solve this problem, we first need to identify which prime numbers are greater than 40. The primes greater than 40 are 41, 43, 47, 53, 59, 61, 67, 71, 73, 79, 83, 89, 97.

Next, we need to check which of these primes satisfies the condition that one plus the prime number is divisible by 3. To do this, we can use the fact that if a number is divisible by 3, then the sum of its digits is also divisible by 3.

Adding 1 to each of these primes, we get:

42 → 4 + 2 = 6 (divisible by 3)
44 → 4 + 4 = 8 (not divisible by 3)
48 → 4 + 8 = 12 (divisible by 3)
54 → 5 + 4 = 9 (divisible by 3)

So the first prime number greater than 40 that satisfies the condition is 43, since 1 + 43 = 44, which is not divisible by 3, but 43 itself is prime. Therefore, the answer is 43.


## EmbeddingRouterChain
Uses embeddings and similarity to route between destination chains. 

[Details](https://python.langchain.com/docs/modules/chains/foundational/router#embeddingrouterchain)

In [ ]:
## Sequential Chain

